<a href="https://colab.research.google.com/github/Rioba-Ian/Deep_Learning/blob/main/mobilenetstreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"riobafelix","key":"5116adfbc4668661906ea8007a47f334"}'}

In [4]:
!mkdir ~/.kaggle

In [5]:
!mv kaggle.json ~/.kaggle

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets list

ref                                                         title                                               size  lastUpdated          downloadCount  
----------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
yamqwe/omicron-covid19-variant-daily-cases                  Omicron daily cases by country (COVID-19 variant)  432KB  2022-01-24 07:15:53           8359  
nkitgupta/jigsaw-regression-based-data                      Jigsaw Regression Based Data                         3GB  2022-01-10 06:29:59            477  
prasertk/netflix-subscription-price-in-different-countries  Netflix subscription fee in different countries      3KB  2022-01-15 07:06:09            812  
yasserh/song-popularity-dataset                             Song Popularity Dataset                            805KB  2022-01-07 13:18:43           1133  
vishalmane10/anime-dataset-2022                             Anime Data

In [9]:
!kaggle datasets download -d alessiocorrado99/animals10

 99% 577M/586M [00:14<00:00, 49.8MB/s]
100% 586M/586M [00:14<00:00, 41.1MB/s]


In [23]:
import os 
import tensorflow as tf 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input 
from tensorflow.keras import callbacks, optimizers 
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator

import numpy as np 
import matplotlib.pyplot as plt 

In [11]:
!ls

animals10.zip  sample_data


In [ ]:
!unzip animals10.zip

In [14]:
from translate import translate

In [15]:
translate

{'butterfly': 'farfalla',
 'cane': 'dog',
 'cat': 'gatto',
 'cavallo': 'horse',
 'chicken': 'gallina',
 'cow': 'mucca',
 'dog': 'cane',
 'elefante': 'elephant',
 'elephant': 'elefante',
 'farfalla': 'butterfly',
 'gallina': 'chicken',
 'gatto': 'cat',
 'mucca': 'cow',
 'pecora': 'sheep',
 'scoiattolo': 'squirrel',
 'spider': 'ragno',
 'squirrel': 'scoiattolo'}

In [17]:

for i in os.listdir("raw-img"):
    try:
        os.rename("raw-img/"+i, "raw-img/"+translate[i])
    except Exception as e:
        print(e)

os.rename("raw-img/ragno", "raw-img/spider" )
# os.rename("raw-img/scoiattolo", "raw-img/squirrel" )

In [18]:
!ls raw-img/

butterfly  cat	chicken  cow  dog  elephant  horse  sheep  spider  squirrel


In [19]:
for i in os.listdir("raw-img"):
    print(i,len(os.listdir("raw-img/"+i)))


horse 2623
dog 4863
squirrel 1862
elephant 1446
chicken 3098
butterfly 2112
cow 1866
sheep 1820
cat 1668
spider 4821


In [20]:
os.mkdir("train")
os.mkdir("test")

for i in os.listdir("raw-img"):
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
    for j in os.listdir("raw-img/"+i)[:1000]:
        os.rename("raw-img/"+i+"/"+j, "train/"+i+"/"+j)
    
    for j in os.listdir("raw-img/"+i)[:400]:
        os.rename("raw-img/"+i+"/"+j, "test/"+i+"/"+j)

    

In [21]:
!ls test

butterfly  cat	chicken  cow  dog  elephant  horse  sheep  spider  squirrel


In [25]:
def img_Data(dir_path, target_size, batch, class_list, preprcess):
    if preprcess:
        gen_object = ImageDataGenerator(preprocessing_function=preprcess)
    else:
        gen_object = ImageDataGenerator()

    return (gen_object.flow_from_directory(dir_path, 
                                   target_size=target_size,
                                   batch_size=batch, 
                                   class_mode='sparse',
                                   classes= class_list,
                                   shuffle=True))

In [27]:
train_data_gen = img_Data("train", (224,224), 500, os.listdir("train"), preprocess_input)
valid_data_gen = img_Data("test", (224,224), 500, os.listdir("test"), preprocess_input)

Found 10000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


In [28]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


9420800/9406464 [==============================] - 0s 0us/step


In [29]:
base_model.trainable = False

In [32]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

In [34]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3,579,978
Trainable params: 1,321,994
Non-trainable params: 2,257,984
_________________________________________________________________


In [35]:
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

save_check = callbacks.ModelCheckpoint('model_wt.h5',save_best_only=True, monitor='val_loss', mode='min')



In [36]:
model.fit(train_data_gen, batch_size=500, validation_data=valid_data_gen, callbacks=[early_stop, save_check], epochs=10)

Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 0.4395 - accuracy: 0.8663 

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


20/20 [==============================] - 397s 19s/step - loss: 0.4395 - accuracy: 0.8663 - val_loss: 0.1608 - val_accuracy: 0.9555
Epoch 2/10
20/20 [==============================] - 386s 20s/step - loss: 0.1239 - accuracy: 0.9636 - val_loss: 0.1534 - val_accuracy: 0.9557
Epoch 3/10
20/20 [==============================] - 375s 19s/step - loss: 0.0872 - accuracy: 0.9747 - val_loss: 0.1412 - val_accuracy: 0.9600
Epoch 4/10
20/20 [==============================] - 421s 21s/step - loss: 0.0595 - accuracy: 0.9835 - val_loss: 0.1410 - val_accuracy: 0.9605
Epoch 5/10
20/20 [==============================] - 389s 20s/step - loss: 0.0405 - accuracy: 0.9914 - val_loss: 0.1434 - val_accuracy: 0.9600
Epoch 6/10
20/20 [==============================] - 376s 19s/step - loss: 0.0313 - accuracy: 0.9928 - val_loss: 0.1476 - val_accuracy: 0.9585
Epoch 7/10
20/20 [==============================] - 381s 19s/step - loss: 0.0205 - accuracy: 0.9967 - val_loss: 0.1472 - val_accuracy: 0.9595
Epoch 8/10
20/20 